<a href="https://colab.research.google.com/github/atbender/training-h2ia/blob/main/sliding_puzzle_uninformed_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Sliding Puzzle Using Uninformed Search

#### Author: Alexandre Thurow Bender

In this notebook, we solve the sliding puzzle game using **Breadth  First Search (BFS)**, **Depth First Search (DFS)**, **Limited DFS**, and **Iterative Deepening**. The objective of this report is to compare the search algorithms with regards to their costs in solving the sliding puzzle task.

In [89]:
import numpy as np
import random
import pandas as pd

We model the problem state space using Nodes. Each Node stores the state, its parent Node, and the directional move that generated the Node's state.

Storing the state is necessary for modelling the board configuration of the puzzle, and for this we use a 2d numpy array.

It is relevant to note the answer to the sliding puzzle problem is not the final state (as it is previously known), but the sequence of movements that take us from the initial configuration to the final state. Storing the parent Node is necessary in order to retrieve the path to the solution, as this implementation does not use an explicit tree data structure but instead uses lists and queues to simulate the tree search traversal.

The Node structure also saves the directional move that generated the state, as this makes it easier to visualize and rebuild the movement sequence solution.

In [90]:
class Node:
    def __init__(self, rows, columns, level=None, parent=None):
        self.parent = parent
        self.move = None
        if level is not None:
            self.level = level
            return
        self.level = self.create_level(rows, columns)

    def create_level(self, rows, columns):
        x = np.arange(0, rows*columns, dtype=int)
        np.random.shuffle(x)
        return x.reshape(rows, columns)

    def __repr__(self):
        return str(self.level)

    def __str__(self):
        return str(self.level)

    def display(self):
        return pd.DataFrame(self.level).style.hide_index().hide_columns()

Generating a random board state is trivial, but it is important to realize not all board states are solvable. We are not interested in generating unsolvable board states as it would hinder the comparison between uninformed searches (or any search, for that matter). 

It would be frustrating if the sliding puzzle game would generate unsolvable states for human players, therefore I assume that would also be the case for search methods.

In [91]:
my_level = Node(3, 5)
my_level.display()

4,10,5,12,14
2,0,11,1,3
6,7,9,8,13


Generating a solvable board instance of the sliding puzzle is harder than checking if a given instance is solvable. For this reason we keep re-generating random instances until the resulting configuration instance is solvable.

Checking for solvability takes into account the parity of the board width, the parity of the total number of inversions in the state map, and the parity of the empty square's row index.

In [92]:
def is_level_solvable(Node):
    level = Node.level
    if is_even(len(level[0])):
        print("level width is even")
        return check_parity_even(level)
    print("level width is odd")
    return check_parity_odd(level)

def check_parity_even(level):
    index_from_below = len(level) - np.where(level == 0)[0].item()
    if is_even(index_from_below):
        # If the grid width is even, and the blank is on an even row counting from the
        # bottom (second-last, fourth-last etc), then the number of inversions in a solvable
        # situation is odd.
        print("index from below is even")
        return not is_even(count_inversions(level))

    # If the grid width is even, and the blank is on an odd row counting from the bottom
    # (last, third-last, fifth-last etc) then the number of inversions in a solvable situation
    # is even.
    print("index from below is odd")
    return is_even(count_inversions(level))

def check_parity_odd(level):
    # If the grid width is odd, then the number of inversions in a solvable situation is even.
    return is_even(count_inversions(level))
    
def count_inversions(level):
    level = level.flatten()
    inversions = 0

    for i in range(0, len(level)):
        for j in range(i + 1, len(level)):
            if level[i] == 0 or level[j] == 0:
                continue
            if level[i] > level[j]:
                inversions += 1
    print(f"level has {inversions} inversions")
    return inversions

def is_even(number):
    return not number % 2

In [94]:
my_level = Node(2, 3)
print(is_level_solvable(my_level))
my_level.display()

level width is odd
level has 7 inversions
False


4,2,5
3,0,1


Below we generate some specific instances to test our check for solvability.

In [95]:
test = Node(2, 3, level=np.array([[5,4,3], [2,1,0]]))
print(is_level_solvable(test))
test.display()

level width is odd
level has 10 inversions
True


5,4,3
2,1,0


In [96]:
test2 = Node(2, 4, np.array([[7,6,5,0], [4,3,2,1]]))
print(is_level_solvable(test2))
test2.level

level width is even
index from below is even
level has 21 inversions
True


array([[7, 6, 5, 0],
       [4, 3, 2, 1]])

In [12]:
test3 = Node(2, 4, level=np.array([[7,6,5,4], [3,2,1,0]]))
print(is_level_solvable(test3))
test3.display()

level width is even
index from below is odd
level has 21 inversions
False


7,6,5,4
3,2,1,0


Creating solvable state instances can be also be done by shuffling a solution with the possible moves. This is arguably more computational intensive, but would garantee the correct initial state generation as we would be restricted to traversing the problem state space.

In [13]:
def create_solvable_level(rows, columns):
    level = Node(rows, columns)
    while not is_level_solvable(level):
        level = Node(rows, columns)
    return level

In [14]:
is_level_solvable(create_solvable_level(2, 3))

level width is odd
level has 3 inversions
level width is odd
level has 3 inversions
level width is odd
level has 6 inversions
level width is odd
level has 6 inversions


True

In [15]:
level = Node(2, 3)

In [16]:
def get_zero_index(level):
    np_index = np.where(level == 0)
    return np_index[0][0], np_index[1][0]

In order to generate neighboring states we must represent the game rules in child state generation. This is done by modelling the possible moves with respect to column and row boundaries.

In [17]:
def generate_neighbors(node):
    zero = get_zero_index(node.level)
    neighbors = []

    # upper bound
    if zero[0] > 0:
        neighbors.append(slide_up(node, zero))
    # left bound
    if zero[1] > 0:
        neighbors.append(slide_left(node, zero))
    # lower bound
    if zero[0] < len(node.level)-1:
        neighbors.append(slide_down(node, zero))
    # right_bound
    if zero[1] < len(node.level[0])-1:
        neighbors.append(slide_right(node, zero))

    return neighbors

The piece slide is implemented with respect to the blank square (zero). Each directional move swaps the blank square with the neighboring number. This is also the moment where the parents and directional moves are saved by the Nodes.

In [18]:
def slide_up(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node.level)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]-1][zero[1]]
    level[zero[0]-1][zero[1]] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "^"
    return generated_node

def slide_left(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]][zero[1]-1]
    level[zero[0]][zero[1]-1] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "<"
    return generated_node

def slide_down(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]+1][zero[1]]
    level[zero[0]+1][zero[1]] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = "v"
    return generated_node

def slide_right(parent_node, zero=None):
    if not zero:
        zero = get_zero_index(parent_node)
    level = parent_node.level.copy()
    level[zero[0]][zero[1]] = level[zero[0]][zero[1]+1]
    level[zero[0]][zero[1]+1] = 0
    generated_node = Node(0, 0, level=level, parent=parent_node)
    generated_node.move = ">"
    return generated_node

In [19]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 2 inversions


0,2,3
1,4,5


In [20]:
neighbors = generate_neighbors(level)

In [21]:
for item in neighbors:
    print(item, "\n")

[[1 2 3]
 [0 4 5]] 

[[2 0 3]
 [1 4 5]] 



To implement any search, we must set up a way of evaluating the states. In uninformed searches, our evaluation is simply checking whether a state is the final state.

In [22]:
def is_solution(node):
    level = node.level
    level = level.flatten()
    if level[-1] != 0:
        return False
    level = level[:-1]
    return np.all(level[:-1] < level[1:])

In [23]:
node = Node(0, 0, level=np.array([[1,2,3], [4,5,0]]))
is_solution(node)

True

In [24]:
node.level

array([[1, 2, 3],
       [4, 5, 0]])

We use a collections deque for implementing a standard queue (FIFO). This is interesting because it is implemented using a linked list, and allows for faster manipulation.

In [25]:
from collections import deque

Breadth First Search expands all node's neighbors before expanding their neighbors. In a sense we are iterating through the layers of the state tree. Because of this the search is complete (because it always finds the solution as long as it exists, and optimal in uniform cases (such as the sliding puzzle). For this same reason, the BFS is extremely memory costly, as it needs to store an exponential amount of neighbors in terms of the branching factor (in the 8-sliding puzzle, the effective branching factor is about 2.6~).

We use a queue for traversing the state space. A state is dequeued, its neighbors generated and enqueued, and the process is repeated until we find the solution. We handle cycles (i.e. repeated states) by storing their hashes, and not expanding those nodes.

In [70]:
def bfs(start_state):
    queue = deque()
    hash_queue = set()
    history = set()
    i = 0

    queue.append((start_state))
    hash_queue.add(hash(str(start_state.level)))

    while True:
        print(f"expanded {i} states")
        i += 1

        state = queue.popleft()
        #print(f"------\n state is: \n{state}\n")
        if hash(str(state.level)) in history:
            continue
        if is_solution(state):
            print(f"solution found in \n{state.level}")
            retrieve_path(state)
            return

        history.add(hash(str(state.level)))
        neighbors = generate_neighbors(state)
        neighbors = [neighbor for neighbor in neighbors if hash(str(neighbor.level)) not in hash_queue and hash(str(neighbor.level)) not in history]

        queue.extend(neighbors)

Retrieving the path and move sequence can be accomplished by storing the parent hierarchy bottom-up from the solution found and inverting it.

In [73]:
def retrieve_path(state):
    path = []
    move_list = []

    while state is not None:
        path.append(state)
        if state.move is not None:
            move_list.append(state.move)
        state = state.parent
    
    print("---path is:---")

    move_list.insert(0, "#")
    print(move_list[::-1])

    for state, move in zip(path[::-1], move_list[::-1]):
        print(f"\n{state.level}")
        print(move)
    
    print(len(move_list))
    return path

In [100]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is odd
level has 1 inversions
level width is even
index from below is odd
level has 3 inversions
level width is even
index from below is even
level has 1 inversions


1,0
3,2


Testing a small instance of the sliding puzzle using BFS.

In [101]:
bfs(level)

expanded 0 states
expanded 1 states
expanded 2 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['v', '#']

[[1 0]
 [3 2]]
v

[[1 2]
 [3 0]]
#
2


In [98]:
!pip install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [99]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


Now we profile the memory and time costs of BFS in practice.

In [102]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is odd
level has 0 inversions


1,2
3,0


In [104]:
%%timeit
%memit bfs(level)

expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found in 
[[1 2]
 [3 0]]
---path is:---
['#']

[[1 2]
 [3 0]]
#
1
peak memory: 231.04 MiB, increment: 0.00 MiB
expanded 0 states
solution found i

This is a small instance, so the time costs are mostly negligible. But do note finding the shortest solution for the n-sliding puzzle is NP-Hard, which means the BFS time costs will increase exponentially regarding the input size.

In [105]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 7 inversions
level width is odd
level has 5 inversions
level width is odd
level has 1 inversions
level width is odd
level has 4 inversions


0,4,1
2,5,3


In [106]:
%%timeit
%memit bfs(level)

expanded 0 states
expanded 1 states
expanded 2 states
expanded 3 states
expanded 4 states
expanded 5 states
expanded 6 states
expanded 7 states
expanded 8 states
expanded 9 states
expanded 10 states
expanded 11 states
expanded 12 states
expanded 13 states
expanded 14 states
expanded 15 states
expanded 16 states
expanded 17 states
expanded 18 states
expanded 19 states
expanded 20 states
expanded 21 states
expanded 22 states
expanded 23 states
expanded 24 states
expanded 25 states
expanded 26 states
expanded 27 states
expanded 28 states
expanded 29 states
expanded 30 states
expanded 31 states
expanded 32 states
expanded 33 states
expanded 34 states
expanded 35 states
expanded 36 states
expanded 37 states
expanded 38 states
expanded 39 states
expanded 40 states
expanded 41 states
expanded 42 states
expanded 43 states
expanded 44 states
expanded 45 states
expanded 46 states
expanded 47 states
expanded 48 states
expanded 49 states
expanded 50 states
expanded 51 states
expanded 52 states
exp

In [131]:
level = create_solvable_level(2, 4)
level.display()

level width is even
index from below is even
level has 6 inversions
level width is even
index from below is even
level has 8 inversions
level width is even
index from below is odd
level has 13 inversions
level width is even
index from below is odd
level has 11 inversions
level width is even
index from below is even
level has 16 inversions
level width is even
index from below is odd
level has 13 inversions
level width is even
index from below is even
level has 15 inversions


3,7,6,0
5,1,4,2


In [132]:
%%timeit
%memit bfs(level)

Streaming output truncated to the last 5000 lines.
expanded 6560 states
expanded 6561 states
expanded 6562 states
expanded 6563 states
expanded 6564 states
expanded 6565 states
expanded 6566 states
expanded 6567 states
expanded 6568 states
expanded 6569 states
expanded 6570 states
expanded 6571 states
expanded 6572 states
expanded 6573 states
expanded 6574 states
expanded 6575 states
expanded 6576 states
expanded 6577 states
expanded 6578 states
expanded 6579 states
expanded 6580 states
expanded 6581 states
expanded 6582 states
expanded 6583 states
expanded 6584 states
expanded 6585 states
expanded 6586 states
expanded 6587 states
expanded 6588 states
expanded 6589 states
expanded 6590 states
expanded 6591 states
expanded 6592 states
expanded 6593 states
expanded 6594 states
expanded 6595 states
expanded 6596 states
expanded 6597 states
expanded 6598 states
expanded 6599 states
expanded 6600 states
expanded 6601 states
expanded 6602 states
expanded 6603 states
expanded 6604 states
expa

In [107]:
level = create_solvable_level(3, 3)
level.display()

level width is odd
level has 20 inversions


8,5,0
4,3,7
2,1,6


In [108]:
%%timeit
%memit bfs(level)

Streaming output truncated to the last 5000 lines.
expanded 87546 states
expanded 87547 states
expanded 87548 states
expanded 87549 states
expanded 87550 states
expanded 87551 states
expanded 87552 states
expanded 87553 states
expanded 87554 states
expanded 87555 states
expanded 87556 states
expanded 87557 states
expanded 87558 states
expanded 87559 states
expanded 87560 states
expanded 87561 states
expanded 87562 states
expanded 87563 states
expanded 87564 states
expanded 87565 states
expanded 87566 states
expanded 87567 states
expanded 87568 states
expanded 87569 states
expanded 87570 states
expanded 87571 states
expanded 87572 states
expanded 87573 states
expanded 87574 states
expanded 87575 states
expanded 87576 states
expanded 87577 states
expanded 87578 states
expanded 87579 states
expanded 87580 states
expanded 87581 states
expanded 87582 states
expanded 87583 states
expanded 87584 states
expanded 87585 states
expanded 87586 states
expanded 87587 states
expanded 87588 states
exp

Observe the time difference from solving a 2x3 instance to solving a 3x3 instance. The state space stored in memory also increases exponentially, note the increasing state number. Despite time being a hard constraint (no one can possibly wait 100 years for a moderately big sliding puzzle solution), the memory constraint can be just as prohibitive in practice as increasing puzzle sizes quickly increase to need several petabytes of memory space.

Depth First Search is implemented in a similar manner, but instead of a queue, we use a stack. This garantees a Node to fully expand one of its neighbors before expanding the others.

DFS is not complete in its canonical form, as it not always finds the solution, particularly because of the existence of cycles in several problems. We handle the existence of cycles by storing the hash of visited board states and not adding them to the stack structure. But even so, DFS is not optimal, as its solution path is often convoluted and significantly bigger than necessary.

The advantage of DFS is it does not store an entire layer of the tree, so it is less memory intensive. It can take quite longer to find relatively shallow solutions in the tree structure (or probably not find them at all).

An improvement we can add to DFS is to add a depth limit to the search. This is also useful in deeper trees (or even infinite trees) to limit the search space and avoid obnoxiously long solutions.

Another notable characteristic of depth limited DFS is it works particularly well in cases where the solution is garanteed to be in a certain depth.

The random state space in a 3x3 instance is 9!. But because exactly half of them are solvable, the actual (non-cyclic) state space is 9!/2. This knowledge can be used to limit uninformed searches.

DFS can also be implemented using recursion, due to its natural recursive traversal of the tree. The recursive approach would need a big stack size, however. This is because each new state would require an additional function call. In a sense, the implementation below is a dynamic programming version of the recursive DFS.

In [123]:
def dfs(start_state, limit):
    queue = deque()
    hash_queue = set()
    history = set()
    i = 0
    level = 1

    queue.append((start_state))
    hash_queue.add(hash(str(start_state.level)))

    while True:
        print(f"expanded {i} states")
        print(f"level is {level}")
        i += 1

        if not len(queue):
            return "cutoff"
        state = queue.pop()

        #print(f"------\n state is: \n{state}\n")

        if hash(str(state.level)) in history:
            level -= 1
            continue
        if is_solution(state):
            print(f"solution found in \n{state.level}")
            retrieve_path(state)
            return

        history.add(hash(str(state.level)))
        neighbors = generate_neighbors(state)
        neighbors = [neighbor for neighbor in neighbors if hash(str(neighbor.level)) not in hash_queue and hash(str(neighbor.level)) not in history]
        if level > limit:
            continue
        level += 1
        queue.extend(neighbors)

In [124]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is odd
level has 0 inversions


1,2
0,3


In [125]:
%%timeit
%memit dfs(level, 5)

expanded 0 states
level is 1
expanded 1 states
level is 2
solution found in 
[[1 2]
 [3 0]]
---path is:---
['>', '#']

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 240.07 MiB, increment: 0.00 MiB
expanded 0 states
level is 1
expanded 1 states
level is 2
solution found in 
[[1 2]
 [3 0]]
---path is:---
['>', '#']

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 240.07 MiB, increment: 0.00 MiB
expanded 0 states
level is 1
expanded 1 states
level is 2
solution found in 
[[1 2]
 [3 0]]
---path is:---
['>', '#']

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 240.07 MiB, increment: 0.00 MiB
expanded 0 states
level is 1
expanded 1 states
level is 2
solution found in 
[[1 2]
 [3 0]]
---path is:---
['>', '#']

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 240.07 MiB, increment: 0.00 MiB
expanded 0 states
level is 1
expanded 1 states
level is 2
solution found in 
[[1 2]
 [3 0]]
---path is:---
['>', '#']

[[1 2]
 [0 3]]
>

[[1 2]
 [3 0]]
#
2
peak memory: 240.07 MiB, increment: 0.00

In [126]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 5 inversions
level width is odd
level has 4 inversions


1,5,0
3,2,4


In [128]:
%%timeit
%memit dfs(level, 10)

expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 5
expanded 5 states
level is 6
expanded 6 states
level is 7
expanded 7 states
level is 8
expanded 8 states
level is 9
expanded 9 states
level is 10
expanded 10 states
level is 11
expanded 11 states
level is 11
expanded 12 states
level is 11
expanded 13 states
level is 11
expanded 14 states
level is 11
expanded 15 states
level is 11
peak memory: 240.07 MiB, increment: 0.00 MiB
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 5
expanded 5 states
level is 6
expanded 6 states
level is 7
expanded 7 states
level is 8
expanded 8 states
level is 9
expanded 9 states
level is 10
expanded 10 states
level is 11
expanded 11 states
level is 11
expanded 12 states
level is 11
expanded 13 states
level is 11
expanded 14 states
level is 11
expanded 15 states
level is 11
p

Note the above DFS execution experienced a cutoff, as it did not find the solution in the depth limit of 100. Let's try it again with increased limit.

In [129]:
%%timeit
%memit dfs(level, 200)

expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 5
expanded 5 states
level is 6
expanded 6 states
level is 7
expanded 7 states
level is 8
expanded 8 states
level is 9
expanded 9 states
level is 10
expanded 10 states
level is 11
expanded 11 states
level is 12
expanded 12 states
level is 13
expanded 13 states
level is 14
expanded 14 states
level is 15
expanded 15 states
level is 16
expanded 16 states
level is 17
expanded 17 states
level is 18
expanded 18 states
level is 19
expanded 19 states
level is 20
expanded 20 states
level is 21
expanded 21 states
level is 22
expanded 22 states
level is 23
expanded 23 states
level is 24
expanded 24 states
level is 25
expanded 25 states
level is 26
expanded 26 states
level is 27
expanded 27 states
level is 28
expanded 28 states
level is 29
expanded 29 states
level is 30
expanded 30 states
level is 31
expanded 31 states
level is 32
expanded 32 states
level is

In [133]:
level = create_solvable_level(2, 4)
level.display()

level width is even
index from below is odd
level has 9 inversions
level width is even
index from below is odd
level has 13 inversions
level width is even
index from below is even
level has 16 inversions
level width is even
index from below is even
level has 11 inversions


7,4,3,0
1,2,5,6


In [134]:
%%timeit
%memit dfs(level, 20_000)

Streaming output truncated to the last 5000 lines.
<

[[4 7 1 6]
 [5 0 3 2]]
^

[[4 0 1 6]
 [5 7 3 2]]
<

[[0 4 1 6]
 [5 7 3 2]]
v

[[5 4 1 6]
 [0 7 3 2]]
>

[[5 4 1 6]
 [7 0 3 2]]
>

[[5 4 1 6]
 [7 3 0 2]]
^

[[5 4 0 6]
 [7 3 1 2]]
>

[[5 4 6 0]
 [7 3 1 2]]
v

[[5 4 6 2]
 [7 3 1 0]]
<

[[5 4 6 2]
 [7 3 0 1]]
<

[[5 4 6 2]
 [7 0 3 1]]
<

[[5 4 6 2]
 [0 7 3 1]]
^

[[0 4 6 2]
 [5 7 3 1]]
>

[[4 0 6 2]
 [5 7 3 1]]
>

[[4 6 0 2]
 [5 7 3 1]]
>

[[4 6 2 0]
 [5 7 3 1]]
v

[[4 6 2 1]
 [5 7 3 0]]
<

[[4 6 2 1]
 [5 7 0 3]]
<

[[4 6 2 1]
 [5 0 7 3]]
<

[[4 6 2 1]
 [0 5 7 3]]
^

[[0 6 2 1]
 [4 5 7 3]]
>

[[6 0 2 1]
 [4 5 7 3]]
>

[[6 2 0 1]
 [4 5 7 3]]
>

[[6 2 1 0]
 [4 5 7 3]]
v

[[6 2 1 3]
 [4 5 7 0]]
<

[[6 2 1 3]
 [4 5 0 7]]
<

[[6 2 1 3]
 [4 0 5 7]]
^

[[6 0 1 3]
 [4 2 5 7]]
>

[[6 1 0 3]
 [4 2 5 7]]
>

[[6 1 3 0]
 [4 2 5 7]]
v

[[6 1 3 7]
 [4 2 5 0]]
<

[[6 1 3 7]
 [4 2 0 5]]
<

[[6 1 3 7]
 [4 0 2 5]]
<

[[6 1 3 7]
 [0 4 2 5]]
^

[[0 1 3 7]
 [6 4 2 5]]
>

[[1 0 3 7]
 [6 4 2 5]]
>

[[1 3 0 7]

Another DFS variant is the iterative deepening DFS, in which we run limited DFS with incrementally deeper layers. This search variant not only is complete, but is now optimal as well. 

The only downside is the necessity of reduntant calculations of the tree. In terms of asymptotic complexity this is not a big problem, as the algorithm is still bounded by the leaves of the tree, that is, reduntant calculations do not elevate the asymptotic complexity of the search. This means we have the best of both worlds, somewhat low memory requirement and low time complexity cost. That being said, it still can take a long time to perform such reduntant calculations.

In [135]:
def i_dfs(start_state):
    i = 1
    while dfs(start_state, i) == "cutoff":
        i += 1

In [138]:
level = create_solvable_level(2, 2)
level.display()

level width is even
index from below is odd
level has 1 inversions
level width is even
index from below is even
level has 3 inversions


0,3
2,1


In [139]:
%%timeit
%memit i_dfs(level)

expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 2
expanded 3 states
level is 2
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 3
expanded 4 states
level is 3
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 4
expanded 5 states
level is 4
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 5
expanded 5 states
level is 5
expanded 6 states
level is 5
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 states
level is 5
expanded 5 states
level is 6
expanded 6 states
level is 6
expanded 7 states
level is 6
expanded 0 states
level is 1
expanded 1 states
level is 2
expanded 2 states
level is 3
expanded 3 states
level is 4
expanded 4 sta

In [140]:
level = create_solvable_level(2, 3)
level.display()

level width is odd
level has 3 inversions
level width is odd
level has 4 inversions


1,5,0
2,4,3


In [141]:
%%timeit
%memit i_dfs(level)

Streaming output truncated to the last 5000 lines.
expanded 235 states
level is 220
expanded 236 states
level is 220
expanded 237 states
level is 220
expanded 238 states
level is 220
expanded 239 states
level is 219
expanded 240 states
level is 220
expanded 241 states
level is 220
expanded 242 states
level is 220
expanded 243 states
level is 219
expanded 244 states
level is 220
expanded 245 states
level is 220
expanded 246 states
level is 220
expanded 247 states
level is 220
expanded 248 states
level is 220
expanded 249 states
level is 220
expanded 250 states
level is 220
expanded 251 states
level is 219
expanded 252 states
level is 220
expanded 253 states
level is 220
expanded 254 states
level is 220
expanded 255 states
level is 220
expanded 256 states
level is 219
expanded 257 states
level is 218
expanded 258 states
level is 217
expanded 259 states
level is 216
expanded 260 states
level is 217
expanded 261 states
level is 218
expanded 262 states
level is 219
expanded 263 states
level

# Conclusion

In the report we explored uninformed searches, including breadth first search, depth first search, limited depth first search, and iterative deepening.

BFS is particularly useful in problems with a mild branching factor, as the state number explosion would be less impairing.

DFS is less memory intensive, but can yield sub-optimal (outright bad) solutions.

Limited DFS is useful to limit the search space and mitigate how bad solutions can get. The solution can, however, be out of the stipulated boundary.

In order to avoid this, one could use iterative deepening, which yields complete and optimal solutions at the cost of reduntant calculations as it runs several bounded DFS. This cost is not asympotically relevant, however.